In [1]:
from salad.t2m import Text2Motion
from salad.utils.get_opt import get_opt

from salad.utils.fixseed import fixseed

import torch
import numpy as np
from os.path import join as pjoin

import os
from os.path import join as pjoin
import torch
import numpy as np
from salad.utils.motion_process import recover_from_ric
from salad.utils.plot_script import plot_3d_motion
from salad.utils.get_opt import get_opt

from salad.data.t2m_dataset import MotionDataset
from torch.utils.data import DataLoader

def plot_t2m(data, text, filename):
    os.makedirs("reconstruction_result", exist_ok=True)
    #data = data[:m_lens[0].item()]
    data = data
    joint = recover_from_ric(torch.from_numpy(data).float(), opt.joints_num).numpy()
    save_path = pjoin("reconstruction_result", f"{filename}.mp4")
    plot_3d_motion(save_path, opt.kinematic_chain, joint, title=text, fps=20)

    np.save(pjoin("reconstruction_result", f"{filename}_pos.npy"), joint)
    np.save(pjoin("reconstruction_result", f"{filename}_feats.npy"), data)

def plot_og(motion_path, text, filename):
    os.makedirs("og_result", exist_ok=True)
    joint = np.load(motion_path)[:260]
    print(joint.shape)
    save_path = pjoin("og_result", f"{filename}.mp4")
    plot_3d_motion(save_path, opt.kinematic_chain, joint, title=text, fps=20)
    np.save(pjoin("og_result", f"{filename}_pos.npy"), joint)
    # np.save(pjoin("og_result", f"{filename}_feats.npy"), )

/opt/conda/envs/salad/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
denoiser_name = "t2m_denoiser_vpred_vaegelu"
dataset_name = "t2m"
model = Text2Motion(denoiser_name, dataset_name)
device = 'cuda:0'

Reading checkpoints/t2m/t2m_denoiser_vpred_vaegelu/opt.txt
Reading checkpoints/t2m/t2m_vae_gelu/opt.txt
Loading VAE Model t2m_vae_gelu
Loading Denoiser Model t2m_denoiser_vpred_vaegelu


0it [00:00, ?it/s]


Loaded CLIP text encoder version ViT-B/32


In [3]:
data_opt = get_opt(model.opt.dataset_opt_path, torch.device('cuda'))
mean = np.load(pjoin(data_opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(data_opt.meta_dir, 'std.npy'))

Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt


In [ ]:
opt = model.vae_opt
data = MotionDataset(opt, mean, std, './dataset/100style/100style.txt')
dataloader = DataLoader(data, batch_size=1, drop_last=True, num_workers=opt.num_workers, shuffle=False, pin_memory=True)

  0%|          | 0/16074 [00:00<?, ?it/s]

100%|██████████| 16074/16074 [00:06<00:00, 2449.49it/s]

Total number of motions 14136, snippets 3037678


In [ ]:
motions = []
for i, batch in enumerate(dataloader):
    batch  = batch.to(device, dtype=torch.float32)
    z, _   = model.vae.encode(batch)
    motion = model.vae.decode(z)
    motions.append(motion)

In [16]:
motion_id = open(f'./dataset/100style/100style.txt').readlines()[0].strip()
src_motion = motions[0].detach().cpu().numpy() * std + mean
plot_t2m(src_motion[0], "recon", "src")
plot_og(f'./dataset/100style/new_joints/{motion_id}.npy', "og", "og")

(260, 22, 3)


In [ ]:
from moviepy.editor import VideoFileClip, clips_array
og_video = VideoFileClip(pjoin("delete", "1.mp4"))
src_video = VideoFileClip(pjoin("delete", "2.mp4"))
final_video = clips_array([[og_video, src_video]])
final_video.write_videofile(pjoin("reconstruction_result", "final.mp4"))

from IPython.display import Video
Video("reconstruction_result/final.mp4", width=800, height=400)

Moviepy - Building video reconstruction_result/final.mp4.
Moviepy - Writing video reconstruction_result/final.mp4



Moviepy - Done !
Moviepy - video ready reconstruction_result/final.mp4


In [109]:
joint1 = np.load('./reconstruction_result/src_pos.npy');

In [110]:
joint2 = np.load(f'./dataset/100style/new_joints/{motion_id}.npy')[:260]